<a href="https://colab.research.google.com/github/hristijanpeshov/SHAP-Explainable-Lexicon-Model/blob/master/notebooks/RoBERTa%20notebooks/auc_model_evaluation_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# User Input

In [ ]:
# enter the location of lexicons (please make sure that there are only lexicons files in the folder)
lexicons_folder_map = {
    'nasdaq': '/content/drive/MyDrive/nasdaq/concatenated datasets/lexicons',
    'fpb': '/content/drive/MyDrive/fpb/concatenated datasets/lexicons',
    'sentfin': '/content/drive/MyDrive/sentfin/concatenated datasets/lexicons'
}

# enter the location of tokenizer
tokenizer_loc = '/content/drive/MyDrive/roberta/roberta_tokenizer'

# enter the location of all evaluation datasets (please make sure that there are only evaluation files in the folder)
eval_datasets_folder_loc = '/content/drive/MyDrive/datasets/evaluation datasets'

# ShapDictModel

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score, matthews_corrcoef
import re
from sklearn.metrics import confusion_matrix
import torch
import nltk
import torch
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

class ShapDictModel:
  def __init__(self, dataset, tokenizer, word_column, category_column, decision_makers, count_column, dataset_source = None):
    self.dataset_source = str.upper(dataset_source) if dataset_source is not None else 'both'
    self.word_column = word_column
    self.tokenizer = tokenizer
    self.count_column = count_column

    # Lemmatizer
    self.lemmatizer = WordNetLemmatizer()

    # source column values
    self.lm_source = 'LM'
    self.tm_source = 'OUR_WORDS'

    # column prefix
    self.lm_prefix = 'LM_'
    self.tm_prefix = 'TM_'

    # category values
    self.positive_category_value = 'positive'
    self.negative_category_value = 'negative'

    # opposite prefix
    self.opposite_prefix = 'opposite_'

    # number of required coefficient when both sources or one sorce
    self.coefficient_number_both_sources = 4
    self.coefficient_number_one_source = 2

    # prefix when one source is chosen
    self.prefix = 'TM_' if self.dataset_source == self.tm_source else 'LM_' if self.dataset_source == self.lm_source else ''

    # source column postfix
    self.source_column = 'src'

    # dataset on which results are calculated
    self.dataset = dataset if self.dataset_source == 'both' else self.extract_dataset_from_source(dataset, self.dataset_source)

    # function that will calculate the score
    self.calculate_score = self.calculate_score_both_dataset_sources if self.dataset_source == 'both' else self.calculate_score_one_dataset_source

    # category when one source is chosen
    self.category = category_column

    # decision makers
    self.decision_makers = decision_makers
    print()
    print(f'Created ShapDictModel with decision makers: {self.decision_makers}')
    print()


  def extract_dataset_from_source(self, dataset, source, only_source_columns=True):
    # depending on which source is chosen, the full dataset will be modified to return the required dataset

    # if LM is chosen as source, then the returned dataset will contain only the words that were originally from the LM dataset with LM_ added as prefix to the columns
    if source == self.lm_source:
      prefix = self.lm_prefix
      column = f'{self.lm_prefix}{self.source_column}'
      opposite_column = f'{self.tm_prefix}{self.source_column}'

      dataset_source = self.lm_source
      opposite_dataset_source = self.tm_source
    else:
      # if OUR_WORDS is chosen as source, then the returned dataset will contain only the words that were originally from our words dataset with TM_ added as prefix to the columns
      prefix = self.tm_prefix
      column = f'{self.tm_prefix}{self.source_column}'
      opposite_column = f'{self.lm_prefix}{self.source_column}'

      dataset_source = self.tm_source
      opposite_dataset_source = self.lm_source

    source_dataset = dataset[(dataset[column] == dataset_source) & ((dataset[opposite_column] == dataset_source) | (dataset[opposite_column] == opposite_dataset_source))]

    # filtering so just the necessary columns will remain
    if only_source_columns:
      columns = list(source_dataset.columns)
      source_columns = [self.word_column] + [column for column in columns if prefix in column]

      return source_dataset[source_columns]

    return source_dataset

  def calculate_score_both_dataset_sources(self, word_occurence, coefficients):
    tm_accumulated_score, tm_opposite_accumulated_score = self.calculate_dataset_source_score(word_occurence, self.tm_prefix)
    lm_accumulated_score, lm_opposite_accumulated_score = self.calculate_dataset_source_score(word_occurence, self.lm_prefix)

    c1, c2, c3, c4 = coefficients

    weighted_score = (c1 * tm_accumulated_score + c2 * tm_opposite_accumulated_score + c3 * lm_accumulated_score + c4 * lm_opposite_accumulated_score)

    return weighted_score

  def calculate_score_one_dataset_source(self, word_occurence, coefficients, prefix):
    accumulated_score, opposite_accumulated_score = self.calculate_dataset_source_score(word_occurence, prefix)

    c1, c2 = coefficients

    weighted_score = (c1 * accumulated_score + c2 * opposite_accumulated_score)

    return weighted_score

  def calculate_decision_score(self, words, coefficients, prefix):
    decision_score = 0
    for word in words:
      # it can only have one occurence, but to check if it occurs at all
      word_occurences = self.dataset.loc[self.dataset[self.word_column] == word].values
      if len(word_occurences) == 0:
        continue

      word_occurence = word_occurences[0]

      decision_score += self.calculate_score_one_dataset_source(word_occurence, coefficients, prefix)

    return decision_score

  def get_coefficients(self, cofficients, source):
    if self.dataset_source == 'both' and source == self.lm_source:
      return cofficients[2:]

    return cofficients[:2]

  def predict_sentence_label(self, sentence, label_t, coefficients):
    tm_words = self.clean_and_lemmatize_words(self.tokenizer.tokenize(sentence))
    lm_words = self.clean_and_lemmatize_words(word_tokenize(sentence))

    decision_score = 0

    both_sources = self.dataset_source == 'both'

    if (self.dataset_source == self.tm_source) or both_sources:
      coefs = self.get_coefficients(coefficients, self.tm_source)
      decision_score += self.calculate_decision_score(tm_words, coefs, self.tm_prefix)

    if (self.dataset_source == self.lm_source) or both_sources:
      coefs = self.get_coefficients(coefficients, self.lm_source)
      decision_score += self.calculate_decision_score(lm_words, coefs, self.lm_prefix)

    label = 1 if decision_score > 0 else 0 if decision_score < 0 else -1

    return label, decision_score

  def __get_wordnet_pos(self, word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

  def clean_and_lemmatize_words(self, words):
    lower_case_words = [str(word).lower().replace('ġ', '').strip() for word in words]
    return [self.lemmatizer.lemmatize(word, self.__get_wordnet_pos(word)) for word in lower_case_words]

  def calculate_word_dm_score(self, word_occurence, decision_maker_column, count_column):
    columns = list(self.dataset.columns)

    decision_maker_index = columns.index(decision_maker_column)

    value = word_occurence[decision_maker_index]

    return value

  def calculate_dataset_source_score(self, word_occurence, column_prefix):
    columns = list(self.dataset.columns)
    category_index = columns.index(f'{column_prefix}{self.category}')

    word_category = word_occurence[category_index]

    selected_category_sign = 1
    opposite_category_sign = 1
    if word_category == self.positive_category_value:
      opposite_category_sign = -1
    elif word_category == self.negative_category_value:
      selected_category_sign = -1

    if selected_category_sign == opposite_category_sign:
      return 0, 0

    opposite_column_prefix = f'{column_prefix}{self.opposite_prefix}'

    selected_category_score = 0
    opposite_category_score = 0
    for decision_maker in self.decision_makers:
      selected_category_score += self.calculate_word_dm_score(word_occurence, f'{column_prefix}{decision_maker}', f'{column_prefix}{self.count_column}') * selected_category_sign
      opposite_category_score += self.calculate_word_dm_score(word_occurence, f'{opposite_column_prefix}{decision_maker}', f'{opposite_column_prefix}{self.count_column}') * opposite_category_sign

    return selected_category_score, opposite_category_score

  def calculate_model_accuracy(self, true_labels, predicted_labels):
    accuracy_indicators = [true_label == predicted_label for true_label, predicted_label in zip(true_labels, predicted_labels)]

    return np.asarray(accuracy_indicators).sum() / len(true_labels)


  def predict_and_evaluate(self, sentences, true_labels, coefficients):
    if self.dataset_source == 'both':
      assert len(coefficients) == self.coefficient_number_both_sources, f'{self.coefficient_number_both_sources} coefficients required, provided {len(coefficients)}'
    else:
      assert len(coefficients) == self.coefficient_number_one_source, f'{self.coefficient_number_one_source} coefficients required, provided {len(coefficients)}'

    xlex_results = [self.predict_sentence_label(sentence, label, coefficients) for sentence, label in zip(sentences, true_labels)]

    predicted_labels = [item[0] for item in xlex_results]
    calculated_scores = [item[1] for item in xlex_results]

    fpr, tpr, thresholds = metrics.roc_curve(true_labels, calculated_scores, pos_label=1)
    auc = metrics.auc(fpr, tpr)

    print(f'AUC score: {auc}')

    return predicted_labels, auc

In [ ]:
def convert_to_num(data, column):
  sentiment_map = {
      'positive': 1,
      'negative': 0
  }
  return data[column].apply(lambda s: sentiment_map[s]).values

In [ ]:
import sys
import pytz

def create_summary_results(lexicon_source, lexicon_datasets, evaluation_datasets, drive_loc):

  evaluation_summary_one_line = []
  for lexicon_name in lexicon_datasets:
    lexicon = lexicon_datasets[lexicon_name]
    lexicon_normalized = 'normalized' in lexicon_name

    for evaluate_dataset_name in evaluation_datasets:
      evaluate_dataset = evaluation_datasets[evaluate_dataset_name]
      sentences = evaluate_dataset['text'].values
      true_labels = evaluate_dataset['sentiment'].values
      coefs_our_words = [0.3, 0.1]
      coefs_lmd = [0.1, 0.5]
      coefs = coefs_our_words + coefs_lmd

      dataset = lexicon.copy(True)

      lexicon_type = 'normalized' if lexicon_normalized else 'merged'
      log_name = f'{lexicon_source}-{lexicon_type}-{evaluate_dataset_name}'
      log_location = f'{drive_loc}/{log_name}.txt'
      f = open(log_location, 'w')

      evalutaion_result_single_line = evaluate(dataset, sentences, true_labels, coefs, coefs_our_words, coefs_lmd, lexicon_source, lexicon_normalized, evaluate_dataset_name, log_location, f)
      evaluation_summary_one_line.append(evalutaion_result_single_line)

      f.close()


  evaluation_summary_one_line_df = pd.DataFrame(evaluation_summary_one_line, columns = ['Lexicon Source', 'Lexicon Normalized', 'Evaluation Dataset', 'Decision Maker', 'C1', 'C2', 'C3', 'C4',
                        'OUR + LMD',	'OUR', 'LMD', 'LMD on LMD', 'OUR on LMD', 'OUR + LMD on LMD', 'Log'])

  evaluation_summary_one_line_df.to_csv(f'{drive_loc}/auc_summary_df.csv', index=False)

  return evaluation_summary_one_line_df

def create_repetition_string(repetitive_string, times, split):
  repetition = [repetitive_string] * times

  return f"{split}".join(repetition)

def write_result_to_file(f, execution_number, results_dictionary, classification_report, confussion_matrix):
  f.write(f'Execution number: {execution_number}')
  f.write(create_repetition_string('\n', 2, ''))

  [f.write(f'{metric}: {results_dictionary[metric]}\n') for metric in results_dictionary]

  f.write(create_repetition_string('\n', 3, ''))

  f.write("Classification Report:\n")
  f.write(classification_report)

  f.write(create_repetition_string('\n', 2, ''))

  f.write("Confusion Matrix:\n")
  f.write(str(confussion_matrix))

  f.write(create_repetition_string('\n', 3, ''))
  f.write(create_repetition_string('-', 70, ''))
  f.write(create_repetition_string('\n', 4, ''))

def end_of_section(f):
  for i in range (4):
    f.write(create_repetition_string('-', 70, ''))
    f.write(create_repetition_string('\n', 1, ''))

  f.write(create_repetition_string('\n', 2, ''))

def create_had_responses_df(sentences, predicted_labels, true_labels):
  lm_responses_df = pd.DataFrame(list(zip(sentences, predicted_labels, true_labels)), columns = ['sentences', 'predicted_label', 'true_label'])

  had_responses_df = lm_responses_df[lm_responses_df['predicted_label'] != -1]

  return had_responses_df

In [ ]:
def run_shap_dict_model(dataset, sentences, true_labels, coefs, dataset_source = None):
  shap_dict_model = ShapDictModel(dataset, tokenizer, 'word', 'category', ['average_shap_values'], 'count', dataset_source = dataset_source)
  result = shap_dict_model.predict_and_evaluate(sentences, true_labels, coefs)

  return result

def evaluation_result_one_line(metrics_list, coefs, lexicon_source, lexicon_name, evaluate_dataset_name, log_location):
  return [lexicon_source, lexicon_name, evaluate_dataset_name, 'average_shap_values'] + coefs + metrics_list + [log_location]


def evaluate(dataset, sentences, true_labels, coefs, coefs_our_words, coefs_lmd, lexicon_source, lexicon_name, evaluate_dataset_name, log_location, f):
  auc_results = []
  f.write('OUR WORDS + LMD')
  f.write(create_repetition_string('\n', 2, ''))
  result = run_shap_dict_model(dataset, sentences, true_labels, coefs)
  f.write(str(result))
  f.write('\n')
  auc_results.append(result[1])

  f.write(create_repetition_string('\n', 2, ''))
  f.write('OUR WORDS')
  f.write(create_repetition_string('\n', 2, ''))
  result = run_shap_dict_model(dataset, sentences, true_labels, coefs_our_words, dataset_source='OUR_WORDS')
  f.write(str(result))
  f.write('\n')
  auc_results.append(result[1])

  f.write(create_repetition_string('\n', 2, ''))
  f.write('LMD')
  f.write(create_repetition_string('\n', 2, ''))
  result = run_shap_dict_model(dataset, sentences, true_labels, coefs_lmd, dataset_source='LM')
  f.write(str(result))
  f.write('\n')
  auc_results.append(result[1])

  f.write(create_repetition_string('\n', 2, ''))

  had_responses_df = create_had_responses_df(sentences, result[0], true_labels)

  f.write('LMD accuracy for those that LMD had answer for')
  f.write(f"LMD accuracy score: {accuracy_score(had_responses_df['true_label'].values, had_responses_df['predicted_label'].values)}")
  result = run_shap_dict_model(dataset, had_responses_df['sentences'].values, had_responses_df['true_label'].values, coefs_lmd, dataset_source='LM')
  f.write(str(result))
  f.write('\n')
  auc_results.append(result[1])

  f.write(create_repetition_string('\n', 2, ''))
  f.write('OUR WORDS accuracy for those that LMD had answer for')
  f.write(create_repetition_string('\n', 2, ''))
  # prediction by using explainable words applied only on the sentences on which LMD had a prediction (there were words in the sentence that were also in LMD)
  result = run_shap_dict_model(dataset, had_responses_df['sentences'].values, had_responses_df['true_label'].values, coefs_our_words, dataset_source='OUR_WORDS')
  f.write(str(result))
  f.write('\n')
  auc_results.append(result[1])

  f.write('\n')
  f.write('OUR WORDS + LMD accuracy for those that LMD had answer for')
  f.write(create_repetition_string('\n', 2, ''))
  # prediction by using explainable words + LMD words, applied again only on the sentences on which LMD had a prediction (there were words in the sentence that were also in LMD)
  result = run_shap_dict_model(dataset, had_responses_df['sentences'].values, had_responses_df['true_label'].values, coefs)
  f.write(str(result))
  f.write('\n')
  auc_results.append(result[1])

  return evaluation_result_one_line(auc_results, coefs, lexicon_source, lexicon_name, evaluate_dataset_name, log_location)

# Evaluate lexicons

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd

def extract_file_name(file_loc):
  return file_loc.split('/')[-1].split('.')[0]

def extract_datasets_map(datasets_location):
  location = datasets_location if datasets_location[-1] == '/' else f'{datasets_location}/'
  files_locations = [join(location, f) for f in listdir(location) if isfile(join(location, f))]

  print(f'Reading datasets from: {location} ...')

  assert files_locations != 0, 'No files found in the provided location'

  datasets_map = {}
  for f in files_locations:
    print(f'Reading dataset: {f} ...')
    dataset = pd.read_csv(f)
    datasets_map[extract_file_name(f)] = dataset

  print(f'Reading datasets successfully finished ...')

  return datasets_map


def create_results_folder(loc):
  parent_location = os.path.abspath(os.path.join(loc, os.pardir))
  mod_location = parent_location if parent_location[-1] == '/' else f'{parent_location}/'

  results_location = f'{mod_location}auc results'

  if not os.path.exists(results_location):
    os.makedirs(results_location)

  print(f'Created results dataset on location: {results_location} ...')

  return results_location

In [ ]:
tokenizer = torch.load(tokenizer_loc)

auc_summary_map = {}

for lex_name in lexicons_folder_map:
  lexicons_loc = lexicons_folder_map[lex_name]

  lexicon_datasets_map = extract_datasets_map(lexicons_loc)

  eval_datasets_map = extract_datasets_map(eval_datasets_folder_loc)

  if lex_name == 'fpb':
    filtered_dict = dict(filter(lambda item: item[0] != 'financial_phrase_bank', eval_datasets_map.items()))
  else:
    filtered_dict = eval_datasets_map

  results_folder_loc = create_results_folder(lexicons_loc)

  auc_summary_map[lex_name] = f'{results_folder_loc}/auc_summary_df.csv'

  print(f'All eval dfs: {filtered_dict.keys()}')
  df = create_summary_results(lex_name, lexicon_datasets_map, filtered_dict, results_folder_loc)

In [ ]:
def change_columns_names(columns):
  return [column.replace('OUR', 'XLex').replace('LMD', 'LM') for column in columns]

In [ ]:
auc_summary_dfs = {}
for lex in auc_summary_map:
  summary_df = pd.read_csv(auc_summary_map[lex])
  summary_df.columns = change_columns_names(summary_df.columns)
  summary_df = summary_df.reindex(columns = list(summary_df.columns[:8]) + ['LM', 'XLex', 'XLex + LM', 'LM on LM', 'XLex on LM', 'XLex + LM on LM'])
  auc_summary_dfs[lex] = summary_df

In [ ]:
# presenting the AUC results for nasdaq lexicon
auc_summary_dfs['nasdaq']

,Lexicon Source,Lexicon Normalized,Evaluation Dataset,Decision Maker,C1,C2,C3,C4,LM,XLex,XLex + LM,LM on LM,XLex on LM,XLex + LM on LM
0,nasdaq,True,fpb_fiqa,average_shap_values,0.3,0.1,0.1,0.5,0.721999,0.759803,0.813897,0.805282,0.757966,0.837189
1,nasdaq,True,financial_phrase_bank,average_shap_values,0.3,0.1,0.1,0.5,0.742708,0.812976,0.847743,0.833128,0.825999,0.878983
2,nasdaq,True,fiqa_labeled_df,average_shap_values,0.3,0.1,0.1,0.5,0.701729,0.766941,0.795834,0.847608,0.820988,0.894290
3,nasdaq,True,sem_eval,average_shap_values,0.3,0.1,0.1,0.5,0.775859,0.780678,0.846006,0.810000,0.811852,0.843210
4,nasdaq,False,fpb_fiqa,average_shap_values,0.3,0.1,0.1,0.5,0.721999,0.762700,0.815333,0.805282,0.756962,0.836269
5,nasdaq,False,financial_phrase_bank,average_shap_values,0.3,0.1,0.1,0.5,0.742708,0.820163,0.850193,0.833128,0.830429,0.881652
6,nasdaq,False,fiqa_labeled_df,average_shap_values,0.3,0.1,0.1,0.5,0.701729,0.761604,0.788524,0.847608,0.834877,0.903549
7,nasdaq,False,sem_eval,average_shap_values,0.3,0.1,0.1,0.5,0.775859,0.787729,0.848862,0.810000,0.808395,0.842469


In [ ]:
# presenting the AUC results for FPB - financial_phrase_bank lexicon
auc_summary_dfs['fpb']

,Lexicon Source,Lexicon Normalized,Evaluation Dataset,Decision Maker,C1,C2,C3,C4,LM,XLex,XLex + LM,LM on LM,XLex on LM,XLex + LM on LM
0,fpb,True,fpb_fiqa,average_shap_values,0.3,0.1,0.1,0.5,0.721999,0.726996,0.795456,0.805282,0.728639,0.823049
1,fpb,True,fiqa_labeled_df,average_shap_values,0.3,0.1,0.1,0.5,0.701729,0.698770,0.777791,0.847608,0.763889,0.879630
2,fpb,True,sem_eval,average_shap_values,0.3,0.1,0.1,0.5,0.775859,0.738710,0.838599,0.810000,0.777284,0.839012
3,fpb,False,fpb_fiqa,average_shap_values,0.3,0.1,0.1,0.5,0.721999,0.734904,0.801920,0.805282,0.739586,0.829464
4,fpb,False,fiqa_labeled_df,average_shap_values,0.3,0.1,0.1,0.5,0.701729,0.702831,0.778139,0.847608,0.771605,0.882716
5,fpb,False,sem_eval,average_shap_values,0.3,0.1,0.1,0.5,0.775859,0.752454,0.847747,0.810000,0.795062,0.841975


In [ ]:
# presenting the AUC results for sentfin - fiqa_fpb_sentfin_neutral lexicon
auc_summary_dfs['sentfin']

,Lexicon Source,Lexicon Normalized,Evaluation Dataset,Decision Maker,C1,C2,C3,C4,LM,XLex,XLex + LM,LM on LM,XLex on LM,XLex + LM on LM
0,sentfin,True,fpb_fiqa,average_shap_values,0.3,0.1,0.1,0.5,0.721999,0.760639,0.810197,0.805282,0.775635,0.837733
1,sentfin,True,financial_phrase_bank,average_shap_values,0.3,0.1,0.1,0.5,0.742708,0.843681,0.876970,0.833128,0.832307,0.875609
2,sentfin,True,fiqa_labeled_df,average_shap_values,0.3,0.1,0.1,0.5,0.701729,0.751102,0.799141,0.847608,0.846451,0.915895
3,sentfin,True,sem_eval,average_shap_values,0.3,0.1,0.1,0.5,0.775859,0.759349,0.837305,0.810000,0.781728,0.824938
4,sentfin,False,fpb_fiqa,average_shap_values,0.3,0.1,0.1,0.5,0.721999,0.756155,0.808235,0.805282,0.768397,0.835599
5,sentfin,False,financial_phrase_bank,average_shap_values,0.3,0.1,0.1,0.5,0.742708,0.845142,0.881009,0.833128,0.822420,0.875081
6,sentfin,False,fiqa_labeled_df,average_shap_values,0.3,0.1,0.1,0.5,0.701729,0.741703,0.792643,0.847608,0.817901,0.901235
7,sentfin,False,sem_eval,average_shap_values,0.3,0.1,0.1,0.5,0.775859,0.744400,0.832485,0.810000,0.775062,0.824938
